In [27]:
from torch_geometric.loader import DataLoader
from p4_save_pytorch_datasets import FIFAGSDataset 
import torch


In [38]:
ROOT = '.'

dataset = FIFAGSDataset(ROOT)

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[1]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

torch.manual_seed(153)
dataset = dataset.shuffle()

train_dataset = dataset[:475]
test_dataset = dataset[475:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()



# # Usage
# dataset = MyDataset(root='path/to/dataset')


# train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


# class GCN(torch.nn.Module):
#     def __init__(self, num_node_features, hidden_channels):
#         super(GCN, self).__init__()
#         self.conv1 = GCNConv(num_node_features, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, hidden_channels)
#         self.lin = torch.nn.Linear(hidden_channels, 1)  # Binary classification

#     def forward(self, data):
#         x, edge_index, batch = data.x, data.edge_index, data.batch
#         x = self.conv1(x, edge_index)
#         x = F.relu(x)
#         x = self.conv2(x, edge_index)
#         x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
#         x = self.lin(x)
#         return x

# model = GCN(num_node_features=1, hidden_channels=16)

# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# def train():
#     model.train()
#     for data in train_loader:
#         optimizer.zero_grad()
#         out = model(data)
#         loss = F.binary_cross_entropy_with_logits(out, data.y.float().unsqueeze(1))
#         loss.backward()
#         optimizer.step()

# for epoch in range(1, 201):
#     train()



Dataset: FIFAGSDataset(568):
Number of graphs: 568
Number of features: 3
Number of classes: 9

Data(x=[19, 3], edge_index=[2, 89], edge_attr=[89, 3], y=[1])
Number of nodes: 19
Number of edges: 89
Average node degree: 4.68
Has isolated nodes: True
Has self-loops: False
Is undirected: False
Number of training graphs: 475
Number of test graphs: 93
Step 1:
Number of graphs in the current batch: 32
DataBatch(x=[600, 3], edge_index=[2, 3396], edge_attr=[3396, 3], y=[32], batch=[600], ptr=[33])

Step 2:
Number of graphs in the current batch: 32
DataBatch(x=[615, 3], edge_index=[2, 3456], edge_attr=[3456, 3], y=[32], batch=[615], ptr=[33])

Step 3:
Number of graphs in the current batch: 32
DataBatch(x=[603, 3], edge_index=[2, 3281], edge_attr=[3281, 3], y=[32], batch=[603], ptr=[33])

Step 4:
Number of graphs in the current batch: 32
DataBatch(x=[611, 3], edge_index=[2, 3124], edge_attr=[3124, 3], y=[32], batch=[611], ptr=[33])

Step 5:
Number of graphs in the current batch: 32
DataBatch(x=[

In [39]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(3, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=9, bias=True)
)


In [40]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5979, Test Acc: 0.6237
Epoch: 002, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 003, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 004, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 005, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 006, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 007, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 008, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 009, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 010, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 011, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 012, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 013, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 014, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 015, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 016, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 017, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 018, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 019, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 020, Train Acc: 0.6000, Test Acc: 0.6237
Epoch: 021, Train Acc: 0.6000, Test Acc:

KeyboardInterrupt: 